In [11]:
import nibabel as nib
import numpy as np
import os

In [13]:
def split_tmap_hemispheres(tmap_path, output_dir):
    # Load the T-map NIfTI file
    img = nib.load(str(tmap_path))
    data = img.get_fdata()
    affine = img.affine
    
    # Get the voxel indices for the middle (x=0) based on the affine
    x_voxel_coords = np.arange(data.shape[0])  # Get voxel indices along x-axis
    x_world_coords = nib.affines.apply_affine(affine, np.column_stack([x_voxel_coords, np.zeros_like(x_voxel_coords), np.zeros_like(x_voxel_coords)]))[:, 0]
    
    # Find the voxel index closest to x=0 in world space
    mid_x_index = np.argmin(np.abs(x_world_coords))

    # Create left hemisphere (erase right, x > 0)
    left_data = data.copy()
    left_data[mid_x_index:, :, :] = 0  # Zero out right hemisphere
    left_img = nib.Nifti1Image(left_data, affine, img.header)
    left_output_path = os.path.join(output_dir, f'{tmap_path.stem}_left.nii.gz')
    nib.save(left_img, left_output_path)

    # Create right hemisphere (erase left, x < 0)
    right_data = data.copy()
    right_data[:mid_x_index, :, :] = 0  # Zero out left hemisphere
    right_img = nib.Nifti1Image(right_data, affine, img.header)
    right_output_path = os.path.join(output_dir, f'{tmap_path.stem}_right.nii.gz')
    nib.save(right_img, right_output_path)

    print(f"Saved left hemisphere T-map to: {left_output_path}")
    print(f"Saved right hemisphere T-map to: {right_output_path}")

# Example usage
from pathlib import Path

second_level = Path('path_to_second_level_directory')  # Replace with actual directory
for tmap in second_level.glob('**/spmT_0001.nii'):
    print(f"Processing: {tmap}")
    split_tmap_hemispheres(tmap, tmap.parent)

In [14]:
from pathlib import Path
second_level =Path('/home/neel/Documents/SPM_results/second_level')
for tmap in second_level.glob('**/spmT_0001.nii'):
    print(tmap)
    split_tmap_hemispheres(tmap, tmap.parent)


/home/neel/Documents/SPM_results/second_level/SPM-A_II_multireg_test (FINAL)/spmT_0001.nii
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_multireg_test (FINAL)/spmT_0001_left.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-A_II_multireg_test (FINAL)/spmT_0001_right.nii.gz
/home/neel/Documents/SPM_results/second_level/SPM-V_II_Zipf_multireg_december_allsubs (FINAL)/spmT_0001.nii
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_Zipf_multireg_december_allsubs (FINAL)/spmT_0001_left.nii.gz
Saved right hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_Zipf_multireg_december_allsubs (FINAL)/spmT_0001_right.nii.gz
/home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/spmT_0001.nii
Saved left hemisphere T-map to: /home/neel/Documents/SPM_results/second_level/SPM-V_II_Lg10BG_multireg_november/spmT_0001_left.nii.gz
Saved right